In [1]:
# Random Imports
import os
import sys
import yaml
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

# Regular schema dictates that we put DATAPATH
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ['WANDB_NOTEBOOK_NAME'] = 'train.ipynb'

from ionpy.util import Config

# Setup direcrtories
root = Path("/storage/vbutoi/scratch/ESE")
scratch_root = Path("/storage/vbutoi/scratch/ESE")
code_root = Path("/storage/vbutoi/projects/ESE")

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

In [2]:
%%yaml default_cfg 

experiment:
  seed: '?' 
    
dataloader:
  batch_size: '?' 
  num_workers: '?' 
  pin_memory: True 

optim: 
  _class: torch.optim.Adam
  lr: 1.0e-4
  weight_decay: 0.0 
  
train:
  epochs: '?' 
  eval_freq: '?' 

# Used for additional data experiment.
data:
  train_splits: train
  val_splits: val
  
loss_func: 
  _class: '?'
  from_logits: True
  batch_reduction: 'mean' 

# loss_func: 
#   classes:
#   - _class: ese.experiment.losses.SoftDiceLoss
#     from_logits: True
#     batch_reduction: 'mean' 
#   - _class: ese.experiment.losses.PixelCELoss
#     from_logits: True
#     batch_reduction: 'mean' 
#   weights:
#   - 1.0
#   - 1.0

<IPython.core.display.Javascript object>

In [3]:
%%yaml model_cfg  

model:
  _class: ese.experiment.models.unet.UNet
  filters: [64, 64, 64, 64, 64]
  convs_per_block: '?' 

<IPython.core.display.Javascript object>

In [4]:
%%yaml callbacks_cfg

callbacks:
  step:
    - ese.experiment.callbacks.ShowPredictions
  epoch:
    - ese.experiment.callbacks.WandbLogger
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.ModelCheckpoint:
        monitor: dice_score
        phase: val

<IPython.core.display.Javascript object>

In [5]:
%%yaml experiment_cfg 

# name: "ACDC_SoftDiceLoss"
# name: "ACDC_PixelCELoss"
# name: "ACDC_SoftDiceLoss_wSVLS"
name: "ACDC_PixelCELoss_wSVLS"

## SVLS 
augmentations:
    train:
        - ese.experiment.augmentation.SVLS:
            ksize: 3
            sigma: 1
            always_apply: True
            include_center: True 

# Stndard segmentation data augmentation.
# augmentations:
#     train:
#         - albumentations.ElasticTransform:
#             alpha: 90 
#             sigma: 7 
#             alpha_affine: 0 
#             p: 0.5
#             always_apply: True
#         - albumentations.Flip:
#             p: 0.5
#         - ese.experiment.augmentation.SVLS: # SVLS
#             ksize: 3
#             sigma: 1
#             always_apply: True
#             include_center: True 

experiment:
    seed: 40
    seed_range: 8

data:
    _class: "ese.experiment.datasets.ACDC"
    label_threshold: 0.5
    version: 0.1

loss_func:
    # _class: ese.experiment.losses.SoftDiceLoss
    _class: ese.experiment.losses.PixelCELoss

<IPython.core.display.Javascript object>

In [6]:
# Local imports
from ese.experiment.analysis.analysis_utils.submit_utils import get_ese_training_configs 

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([model_cfg, callbacks_cfg])

# Get the different experiment cfg yamls.
train_cfgs = get_ese_training_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg
)

In [7]:
len(train_cfgs)

8

## Running Jobs

In [8]:
# ######## FOR DEBUGGIN
# from ese.experiment.experiment import run_ese_exp, CalibrationExperiment

# run_ese_exp(
#     config=train_cfgs[0], 
#     experiment_class=CalibrationExperiment,
#     gpu='0',
#     # gpu='4',
#     run_name='debug',
#     show_examples=True,
#     track_wandb=False
# )

In [9]:
# FOR SUBMISSION
from ese.experiment.experiment import submit_ese_exps, CalibrationExperiment 

submit_ese_exps(
    config_list=train_cfgs,
    experiment_class=CalibrationExperiment,
    # available_gpus=['0', '1', '2', '3'],
    available_gpus=['4', '5', '6', '7'],
    track_wandb=True
)

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


Submitted job id: 1073739 on gpu: 4.
Submitted job id: 1074248 on gpu: 5.
Submitted job id: 1074367 on gpu: 6.
Submitted job id: 1074640 on gpu: 7.
Submitted job id: 1074950 on gpu: 4.
Submitted job id: 1075334 on gpu: 5.
Submitted job id: 1076076 on gpu: 6.
Submitted job id: 1076374 on gpu: 7.
